In [132]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 
from typing import List, Tuple, Dict, Optional, Union

import qube
%qubed

%load_ext autoreload
%autoreload 2
from qube import lookup
from qube.core.basics import Position, ZERO_COSTS, TransactionCostsCalculator
from qube.core.strategy import Event, Instrument

import tests.qube.ta.utils_for_testing as test
from dataclasses import dataclass

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from qube.core.series import TimeSeries; from qube.ta.indicators import sma; del sys.modules['qube.core.series']; del sys.modules['qube.ta.indicators']
from qube.core.series import TimeSeries, OHLCV, time_as_nsec, Quote, Trade
from qube.ta.indicators import ( sma, ema, tema, dema, kama, highest, lowest)

# Position

In [184]:
qs = [
    Quote('2024-01-01 12:00:00', 45000, 45000.5, 100, 50),
    Trade('2024-01-01 12:01:00', 45010, 10, 1),
    Trade('2024-01-01 12:02:00', 45015, 10, 1),
    Quote('2024-01-01 12:03:00', 45020, 45021, 0, 0),
    Quote('2024-01-01 12:04:00.100', 45120, 45121, 0, 0),
]
tcc = TransactionCostsCalculator(0.04/100, 0.04/100)

In [202]:
i, s = lookup['BINANCE:BTCUSDT'][0], 1
i, s = lookup['BINANCE.UM:BTCUSDT'][0], 100
# i, s = lookup['BINANCE.CM:BTCUSD_PERP'][0], 1
p = Position(i, tcc)
p.update_market_price(qs[0]); print(p)
p.update_position(time_as_nsec('2024-01-01 12:00:30'), s, 45010); print(p, '<-')
for q in qs[1:]:
    p.update_market_price(q); print(p)
print(p.commissions)

[BINANCE.UM:BTCUSDT] 2024-01-01 12:00:00.000000      0.0 45000.5     0.00 $0.00 / 0.0
[BINANCE.UM:BTCUSDT] 2024-01-01 12:00:30.000000    100.0 45010.0 4500900.00 $4501000.00 / 100.0 <-
[BINANCE.UM:BTCUSDT] 2024-01-01 12:01:00.000000    100.0 45010.0 4500900.00 $4501000.00 / 100.0
[BINANCE.UM:BTCUSDT] 2024-01-01 12:02:00.000000    100.0 45015.0 4501400.00 $4501500.00 / 100.0
[BINANCE.UM:BTCUSDT] 2024-01-01 12:03:00.000000    100.0 45020.0 4501900.00 $4502000.00 / 100.0
[BINANCE.UM:BTCUSDT] 2024-01-01 12:04:00.100000    100.0 45120.0 4511900.00 $4512000.00 / 100.0
0.04


# Strategy high level design

In [55]:
Quote('2020-01-01 12:00:01.22', 12.3, 12.4, 0, 0)
Trade('2020-01-01 12:00:01.22', 12.3, 10000, 0)

[2020-01-01T12:00:01.220000000]	12.30000 (10000.0) <make>

In [3]:
class IStrategy:
    def __init__(self, **kwargs) -> None:
        print("Here")
        self.populate_parameters(**kwargs)

    def populate_parameters(self, **kwargs):
        for k,v in kwargs.items():
            if k.startswith('_'):
                raise ValueError("Internal variable can't be set from external parameter !")
            if hasattr(self, k):
                self.__dict__[k] = v

    def instruments(self) -> List[Instrument]:
        pass

    def position(self, position: float, group=None, stop=None, take=None, limit=None, cancel=False):
        print(f"NEW POSITION: {position} [{group}]")
        pass

    def cancel(self, group=None):
        print(f"CANCEL: all from [{group}]")

    def open_position(self, instrument: Instrument) -> float:
        """
        Returns open position
        """
        return 0.0
        
class PositionTracker:
    pass
class CapitalManager:
    pass
class FixedCapitalPercentage(CapitalManager):
    pass
class AtrTracker(PositionTracker):
    pass

In [32]:
class CrossTestClassical(IStrategy):
    period_fast: int = 5
    period_slow: int = 50

    def fit(self):
        for i in self.Instruments():
            ohlc = self.ohlc(i, '1H')
            # do the model trainig stuff ...
        return self

    def process_data(self, timestamp: np.datetime64, instrument: Optional[Instrument]):
        ohlc = self.ohlc(instrument, '1H')
        # tick_bars = self.tickbars(instrument, 200) # returns series of 200 tick bars
        # trades = self.trades(instrument) # returns series of trades
        # quotes = self.quotes(instrument) # returns series of quotes
        fast = ema(ohlc.close, self.period_fast)
        slow = ema(ohlc.close, self.period_slow)
        bar:Bar = ohlc[0]

        if (fast[0] > slow[0]) and (fast[1] < slow[1]):
            return instrument.position(+1, 'long', stop=bar.low, cancel=True)
        elif (fast[0] < slow[0]) and (fast[1] > slow[1]):
            return instrument.position(-1, 'short', stop=bar.high, cancel=True)

    def tracker(self, instrument: Instrument) -> Optional[PositionTracker]:
        return None

    def capital_manager(self) -> Optional[CapitalManager]:
        return None 

In [ ]:
# CrossTestClassical()

In [9]:
class PairsTrading(IStrategy):
    period: int = 15
    timeframe: str = '1H'
    __pair_for: Dict[Instrument, Instrument] = {}

    def fit(self, instrument: Instrument):
        closes = self.prices(self.timeframe, 'close', self.instruments()).pd()
        # - find pairs
        self.pairs_finder(closes, self.period, self.instruments())
        return self

    def pairs_finder(self, period, closes: pd.DataFrame):
        self.__pair_for.clear()
        self.__pair_for[self.instruments()[0]] = self.instruments()[1]

    def process_data(self, timestamp: np.datetime64, instrument1: Optional[Instrument]):
        instrument2 = self.__pair_for[instrument1]
        delta = self.ohlc(instrument1, self.timeframe).close - self.ohlc(instrument2, self.timeframe).close, 
        spread = (delta - ema(delta, self.period)) / std(delta - ema(delta, self.period)) 
        if spread[0] > 2:
            return instrument1.position(-1, 'short'), instrument2.position(+1, 'long')
        elif spread[0] < -2:
            return instrument2.position(-1, 'short'), instrument1.position(+1, 'long')

    def tracker(self, instrument: Instrument) -> Optional[PositionTracker]:
        return None

    def capital_manager(self) -> Optional[CapitalManager]:
        return None 

In [ ]:
s1 = CrossTestClassical(period_fast=444)

In [ ]:
m = Market('binance.um.usdt.vip0')

In [31]:
data1Min = OHLCV('15Min')

In [ ]:
m.backtest(
    {
        'Signals': CrossTestClassical(period_fast=12),
        'CapPct25%': CrossTestClassical(period_fast=12) + FixedCapitalPercentage(capital_in_risk=0.25),
        'AtrTracker2': CrossTestClassical(period_fast=12) + AtrTracker(stop_atrs=2, initial_bar=0, period=200),
    },
    capital_base=100000,
    leverage=5,
    start='2021-01-01', end='now',
    fit_start='2021-01-01', fit_end='now',

    start='2021-01-01', end='',
    fit_start='2021-01-01', fit_end='+4w',

    data = data1Min.resample('15Min'),   # data to feed
    handle_on='closes',                     # default on data's timeframe close prices
    handle_on='closes.1H',                  # on 1H bar's close price
    handle_on='quotes',                     # on every quote
    handle_on='trades',                     # on every trade
    handle_on='trades,quotes,15Min.closes', # on every trade or quote or close
    handle_on='59Min59S',                   # on timer every T
    symbols=['BTCUSDT', 'ETHUSDT', 'ETHUSDT']
)

In [ ]:
'1H - 5S'

In [ ]:
# # from zipline
# result = run_algorithm(start=start.tz_localize('UTC'),
#                        end=end.tz_localize('UTC'),
#                        initialize=initialize,
#                        handle_data=handle_data,
#                        capital_base=100000,
#                        benchmark_returns=benchmark_returns,
#                        bundle='quandl',
#                        data_frequency='daily')